In [37]:
#Purpose: explore my network
import uuid
import networkx as nx
import numpy as np

In [66]:
class Wire:
    def __init__(self, router1, router2):
        # connection between two routers
        self.router1 = router1
        self.router2 = router2
        # list of packets on wire
        # acting as input buffer
        self.packets = []

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return (self.router1 == other.router1) and (self.router2 == other.router2)
    
    def __repr__(self):
        return f"{self.router1.id},{self.router2.id}\n"

    def find_packet(self, packet):
        for p in self.packets:
            if p == packet:
                return p

        return None

    # Queue behaviour
    def insert_packet(self, packet):
        self.packets.append(packet)

    def remove_packet(self, packet, dst):
        p = self.find_packet(packet)

        if dst is self.router1:
            self.hop(p, self.router1)
        elif dst is self.router2:
            self.hop(p, self.router2)

    def hop(self, packet, dst):
        # if we successfully hopped to router, remove packet
        if packet and dst.insert_packet(packet):
            packet.push_to_router(dst)
            self.packets.remove(packet)
            return self.dst

        # operation failure, dst router congested
        return -1


In [73]:
class Router:
    def __init__(self, id, connections, kind, graph):
        # node on graph
        self.id = id
        self.network = graph
        self.kind = kind

        # edges where this is the src
        # list of wires connected to routers
        self.connections = []

        # how many actions are possible in router
        if connections:
          self.actions = len(connections)

        # depending on type, set buffer size
        if kind == 'T':
            self.buffer_size = 100  # change this to hold more packets
        elif kind == 'M':
            self.buffer_size = 25  # change this to hold more packets
        elif kind == 'C' or kind == 'CP':
            self.buffer_size = 10

        self.buffer = []  # Queue

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id == other.id

    def __repr__(self):
        
        return f"id: {self.id}, type: {self.kind}\n connections: {self.connections}\n"

    def is_full(self):
        return len(self.buffer) >= self.buffer_size

    def has_connection(self, dst):
        for connection in self.connections:
            if connection == dst:
                return connection

        return None

    def add_connections(self, wires):
        for wire in wires:
            if self is wire.router1 or self is wire.router2:
                self.connections.append(wire)


    # Queue behaviour
    # todo: if this is the packet dst router, call network to remove packet from transmission
    def insert_packet(self, packet):
        if not self.is_full():
            self.buffer.append(packet)
            return True

        return False

    # push to wire
    def remove_packet(self, dst):
        # if there is a connection to the destination
        # and the buffer is not empty
        # move packet to wire
        wire = self.has_connection(dst)
        if wire and self.buffer:
            packet = self.buffer[0]
            self.buffer = self.buffer[-1]
            packet.push_to_wire(wire, dst)


In [71]:
class Packet:
    def __init__(self, src, dst, graph, path=[]):
        self.id = str(uuid.uuid4())
        self.curr = src
        self.next_hop = None
        self.src = src
        self.dst = dst
        self.path = path
        self.graph = graph

    # add an equal method to find
    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id == other.id

    def on_wire(self):
        return isinstance(self.curr, Wire)

    def on_router(self):
        return isinstance(self.curr, Router)

    def push_to_wire(self, wire, dst):
        if isinstance(self.curr, Router):
            self.curr = wire
            self.next_hop = dst

    def push_to_router(self, router):
        if isinstance(self.curr, Wire):
            self.curr = router
            self.update_path()
            self.next_hop = self.find_next_hop()

    def find_next_hop(self):
        # we are at destination
        if len(self.path) < 2:
            return -1
        # shortest path
        return self.path[1]

    def complete(self):
        return self.curr == self.dst

    def update_path(self):
        self.path = nx.shortest_path(self.graph, self.curr, self.dst)


In [76]:
G = nx.random_internet_as_graph(n=25)
nx.set_node_attributes(G, {})

all_nodes_data = np.array(G.nodes.data())

all_connections = np.array(G.edges())

#create router for each node
#create a wire for each connection
# then build the network
routers = []
wires = []

#building the network
#extract data from nodes and edges and map
for node in all_nodes_data:
  id = node[0]
  kind = node[1]['type']
  routers.append(Router(id=id,connections=None, kind=kind, graph=G))

for connection in all_connections:
  router_id_1 = connection[0]
  router_id_2 = connection[1]
  wire = Wire(routers[router_id_1], routers[router_id_2])
  wires.append(wire)

for router in routers:
    router.add_connections(wires)

print(routers)

[id: 0, type: T
 connections: [0,1
, 0,2
, 0,3
, 0,4
, 0,5
, 0,7
]
, id: 1, type: T
 connections: [0,1
, 1,2
, 1,3
, 1,4
, 1,8
, 1,9
, 1,15
, 1,18
]
, id: 2, type: T
 connections: [0,2
, 1,2
, 2,3
, 2,4
, 2,6
]
, id: 3, type: T
 connections: [0,3
, 1,3
, 2,3
, 3,4
, 3,5
, 3,9
, 3,19
]
, id: 4, type: T
 connections: [0,4
, 1,4
, 2,4
, 3,4
]
, id: 5, type: M
 connections: [0,5
, 3,5
, 5,9
, 5,10
, 5,11
, 5,20
, 5,24
, 5,8
, 5,6
]
, id: 6, type: M
 connections: [2,6
, 5,6
, 6,7
, 6,12
, 6,13
, 6,14
, 6,17
, 6,8
]
, id: 7, type: M
 connections: [0,7
, 6,7
, 7,23
, 7,8
]
, id: 8, type: M
 connections: [1,8
, 5,8
, 6,8
, 7,8
, 8,16
, 8,21
, 8,22
]
, id: 9, type: CP
 connections: [1,9
, 3,9
, 5,9
]
, id: 10, type: C
 connections: [5,10
]
, id: 11, type: C
 connections: [5,11
]
, id: 12, type: C
 connections: [6,12
]
, id: 13, type: C
 connections: [6,13
]
, id: 14, type: C
 connections: [6,14
]
, id: 15, type: C
 connections: [1,15
]
, id: 16, type: C
 connections: [8,16
]
, id: 17, type: C
 

In [61]:
all_connections

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  9],
       [ 0, 12],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  8],
       [ 1,  9],
       [ 2,  3],
       [ 2,  4],
       [ 2,  5],
       [ 2,  7],
       [ 2,  8],
       [ 2, 18],
       [ 3,  4],
       [ 3,  5],
       [ 3,  7],
       [ 3, 10],
       [ 3, 14],
       [ 3, 19],
       [ 3, 23],
       [ 3, 24],
       [ 4,  5],
       [ 4,  7],
       [ 6, 13],
       [ 6, 15],
       [ 6, 16],
       [ 6,  8],
       [ 6,  9],
       [ 6,  7],
       [ 7, 22],
       [ 7,  9],
       [ 7,  8],
       [ 8, 10],
       [ 8, 17],
       [ 8, 20],
       [ 8,  9],
       [ 9, 11],
       [ 9, 21]])

T
T
T
T
T
M
M
M
M
CP
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
